# Clustering Crypto

In [189]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [190]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

coinlist_json = requests.get(url).json()

In [191]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

crypto_df = pd.DataFrame(data = coinlist_json['Data']).T
crypto_df.head(3)

,Algorithm,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,DecimalPlaces,FullName,FullyPremined,...,Rating,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinSupply,TotalCoinsFreeFloat,TotalCoinsMined,Url
42,Scrypt,195131,0,0,N/A,42 Coin,1427211129,0,42 Coin (42),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",42,N/A,42,/coins/42/overview
300,N/A,0,0,0,7605,300 token,1517935016,18,300 token (300),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,N/A,300,/coins/300/overview
365,X11,NaN,NaN,NaN,N/A,365Coin,1480032918,0,365Coin (365),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",2300000000,N/A,NaN,/coins/365/overview


In [192]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
# Create a DataFrame

### Data Preprocessing

In [193]:
# Keep only necessary columns: 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.25698e+06,532000000
433,433 Token,N/A,False,N/A,1.12518e+08,1000000000


In [194]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.25698e+06,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000


In [195]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.25698e+06,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0,0


In [196]:
# Remove the "IsTrading" column
crypto_df.drop(columns=['IsTrading'], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.25698e+06,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0,0


In [197]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.25698e+06,532000000
808,808,SHA-256,PoW/PoS,0,0
1337,EliteCoin,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000


In [198]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.25698e+06,532000000
1337,EliteCoin,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000
XPY,PayCoin,SHA-256,PoS,1.19953e+07,12500000


In [199]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[
    (crypto_df['Algorithm'] != 'N/A') & 
    (crypto_df['ProofType'] != 'N/A') &
    (crypto_df['TotalCoinsMined'] != 'N/A') & 
    (crypto_df['TotalCoinSupply'] != 'N/A')]

crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.25698e+06,532000000
1337,EliteCoin,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000
XPY,PayCoin,SHA-256,PoS,1.19953e+07,12500000


In [157]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinName_df = pd.DataFrame({'CoinName': crypto_df['CoinName']}) 
coinName_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTCD,BitcoinDark
XPY,PayCoin


In [158]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=['CoinName'], inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,42,42
404,Scrypt,PoW/PoS,1.25685e+06,532000000
1337,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,SHA-256,PoW/PoS,1288862,22000000
XPY,SHA-256,PoS,1.19953e+07,12500000


In [159]:
# Create dummy variables for text features
crypto_df_encoded = pd.get_dummies(crypto_df, columns=['Algorithm','ProofType'])
crypto_df_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,42,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.25685e+06,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.94803e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTCD,1288862,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XPY,1.19953e+07,12500000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
# Standardize data
# crypto_scaled = StandardScaler().fit_transform(crypto_df[['TotalCoinsMined', 'TotalCoinSupply']])
crypto_scaled = StandardScaler().fit_transform(crypto_df_encoded)
print(crypto_scaled[0:5])

[[-0.12343301 -0.16083139 -0.04163054 -0.04163054 -0.05892557 -0.05892557
  -0.04163054 -0.08347839 -0.04163054 -0.05892557 -0.07223151 -0.04163054
  -0.04163054 -0.07223151 -0.17407766 -0.04163054 -0.04163054 -0.08347839
  -0.04163054 -0.04163054 -0.14560705 -0.08347839 -0.04163054 -0.04163054
  -0.04163054 -0.04163054 -0.04163054 -0.15168668 -0.04163054 -0.04163054
  -0.16322679 -0.04163054 -0.04163054 -0.08347839 -0.04163054 -0.04163054
  -0.04163054 -0.04163054 -0.04163054 -0.05892557 -0.04163054 -0.04163054
  -0.07223151 -0.09341308 -0.05892557 -0.04163054 -0.04163054 -0.13268622
  -0.05892557 -0.12576654 -0.14560705 -0.04163054 -0.04163054 -0.04163054
  -0.07223151 -0.18932061 -0.04163054 -0.04163054 -0.04163054 -0.04163054
  -0.07223151 -0.18932061 -0.04163054 -0.31441901 -0.04163054 -0.09341308
  -0.07223151 -0.05892557  1.50517    -0.04163054 -0.04163054 -0.05892557
  -0.04163054 -0.05892557 -0.04163054 -0.04163054 -0.04163054 -0.05892557
  -0.04163054 -0.05892557 -0.38317802 

### Reducing Dimensions Using PCA

In [203]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [206]:
# Create a DataFrame with the principal components data
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

crypto_pca_df = pd.DataFrame(
    data=crypto_pca, 
    columns=["principal component 1", "principal component 2", "principal component 3"]    
)
crypto_pca_df.head()

,principal component 1,principal component 2,principal component 3
0,-0.360639,0.110493,0.058617
1,-0.356624,0.110286,0.058609
2,2.427742,0.426013,0.055207
3,-0.319089,0.134647,0.025332
4,-0.273053,0.299787,0.028316


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values


# Create the Elbow Curve using hvPlot


Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
